## 데이터 전처리

- Null값 처리
- 불필요한 피처 제거
- Encoding 등

In [1]:
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df


In [2]:
# 원본 데이터를 재로딩 하고, feature데이터 셋과 Label 데이터 셋 추출. 
import pandas as pd
titanic_df = pd.read_csv('./titanic_train.csv')

y = titanic_df['Survived']
X= titanic_df.drop('Survived',axis=1)

X = transform_features(X)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y,test_size=0.2, random_state=11)

데이터 전처리를 수행하는 tranfrom_features() 함수로 원본데이터 가공

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 결정트리, Random Forest, 로지스틱 회귀를 위한 사이킷런 Classifier 클래스 생성
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression()

# DecisionTreeClassifier 학습/예측/평가
dt_clf.fit(X_train , y_train)
dt_pred = dt_clf.predict(X_test)
print('DecisionTreeClassifier 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))

# RandomForestClassifier 학습/예측/평가
rf_clf.fit(X_train , y_train)
rf_pred = rf_clf.predict(X_test)
print('RandomForestClassifier 정확도:{0:.4f}'.format(accuracy_score(y_test, rf_pred)))

# LogisticRegression 학습/예측/평가
lr_clf.fit(X_train , y_train)
lr_pred = lr_clf.predict(X_test)
print('LogisticRegression 정확도: {0:.4f}'.format(accuracy_score(y_test, lr_pred)))

DecisionTreeClassifier 정확도: 0.7877
RandomForestClassifier 정확도:0.8547
LogisticRegression 정확도: 0.8492


C:\Users\pc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


랜덤 포레스트가 가장 정확도가 높게 나왔는데 교재는 왜 로지스틱이 가장 높은지... 내가 뭘 잘못했나?...
크흠... 

# 교차검증

- model_selection패키지의 KFold 클래스, cross_val_score, GridSeachCV

## KFold 교차검증 수행

In [5]:
from sklearn.model_selection import KFold

def kfold(clf,folds=5):
    # 폴드세트 5개인 KFold 객체 생성, 폴드 수만큼 예측결과 저장을 위한 리스트 생성
    kfold = KFold(n_splits=folds)
    scores=[]
    
    #KFold 교차검증 수행
    for iter_count, (train_index, test_index) in enumerate(kfold.split(X)):
       
        #X 데이터에서 교차검증별 학습과 검증 데이터를 가리키는 index 생성
        X_train, X_test = X.values[train_index],X.values[test_index]
        y_train, y_test = y.values[train_index],y.values[test_index]
        
        # classfier 학습, 예측, 정확도 계산
        clf.fit(X_train,y_train)
        pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test,pred)
        scores.append(accuracy)
        print('교차검증 {0}, 정확도 :{1:.4f}'.format(iter_count,accuracy))
        
    # 5개의 평균정확도 계산
    mean_score = np.mean(scores)
    print('평균정확도 :{0:.4f}'.format(mean_score))

# kfold 호출
kfold(dt_clf,folds=5)

교차검증 0, 정확도 :0.7542
교차검증 1, 정확도 :0.7809
교차검증 2, 정확도 :0.7865
교차검증 3, 정확도 :0.7697
교차검증 4, 정확도 :0.8202
평균정확도 :0.7823


## cross_val_score() 를 이용한 검증

In [6]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(dt_clf, X, y, cv=5)
for iter_count, accuracy in enumerate(scores):
    print('교차검증 {0}, 정확도:{1:4f}'.format(iter_count,accuracy))
print('평균정확도:{0:4f}'.format(np.mean(scores)))

교차검증 0, 정확도:0.743017
교차검증 1, 정확도:0.775281
교차검증 2, 정확도:0.792135
교차검증 3, 정확도:0.786517
교차검증 4, 정확도:0.842697
평균정확도:0.787929


## GridSearchCV()를 이용한 검증

In [11]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth': [2,3,5,10],'min_samples_split':[2,3,5], 'min_samples_leaf':[1,5,8]}

grid_dclf = GridSearchCV(dt_clf, param_grid=parameters, scoring = 'accuracy', cv=5)
grid_dclf.fit(X_train,y_train)

print('GridSearchCV 최적의 하이퍼 파라미터:',grid_dclf.best_params_)
print('GridSearchCV 최고 정확도{0:.4f}:'.format(grid_dclf.best_score_))

# GridSearchCV 최적의 하이퍼 파라미터로 학습된 Estimator로 학습 및 예측 수행
pred = grid_dclf.predict(X_test)
accuracy = accuracy_score(y_test,pred)
print('test 세트에서 DecisionTreeClassifier 정확도 :{0:.4f}'.format(accuracy))


GridSearchCV 최적의 하이퍼 파라미터: {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
GridSearchCV 최고 정확도0.7992:
test 세트에서 DecisionTreeClassifier 정확도 :0.8715
